# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries

from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])

import re
import numpy as np
import pandas as pd
import pickle
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier

from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', con=engine)

X = df['message'].values
y = df.drop(['message', 'original', 'genre'], axis=1).values

In [23]:
type(X)

numpy.ndarray

In [24]:
type(y)

numpy.ndarray

In [25]:
y.related.unique()

AttributeError: 'numpy.ndarray' object has no attribute 'related'

In [26]:
>>> y.shape

(10223, 37)

In [27]:
>>> X.shape

(10223,)

In [28]:
np.isnan(y.values.any())

AttributeError: 'numpy.ndarray' object has no attribute 'values'

### 2. Write a tokenization function to process your text data

In [29]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex , text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
        
    # Convert text to lowercase and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize words
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    # load stop words
    stop_words = stopwords.words("english")
    
    # lemmatize andremove stop words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word).strip() for word in tokens if word not in stop_words]

    return tokens

In [30]:
# initialize tf-idf vectorizer object
vectorizer = CountVectorizer(tokenizer=tokenize)


In [31]:
# compute bag of word counts and tf-idf values
X_vectorized = vectorizer.fit_transform(X.head())

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [32]:
vectorizer.vocabulary_

AttributeError: 'CountVectorizer' object has no attribute 'vocabulary_'

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [33]:
#pipeline = Pipeline([
    #('vect', CountVectorizer(tokenizer = tokenize)),
    #('tfidf', TfidfTransformer()),
    #('clf', RandomForestClassifier()
    # )])

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [ ]:
pipeline.get_params()

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
#pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
for col in range(36):
    print(y_test.columns[col])
    print(classification_report(y_test.iloc[:,col], y_pred[:,col]))
    print('-----------------------------------------------------')

In [ ]:
target_names = list(y.columns)
print(target_names)

In [ ]:
# it performed two tests, On the second one it usually

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
pipeline.get_params().keys()

In [17]:
X.columns

AttributeError: 'Series' object has no attribute 'columns'

In [35]:
parameters = { 'vect__ngram_range': ((1, 1), (1, 2)), 
              #'vect__max_df': (0.5, 1.0), 
              #'vect__max_features': (None, 5000), 
              'tfidf__use_idf': (True, False), 
              'clf__estimator__n_estimators': [10, 20], 
              'clf__estimator__min_samples_split': [2, 3]} 


cv = GridSearchCV(pipeline, param_grid=parameters, scoring='f1_micro', verbose=1, n_jobs=-1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [36]:
cv.fit(X_train, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


ValueError: multiclass-multioutput is not supported

In [ ]:
y_pred = cv.predict(X_test)

In [ ]:
class_report = classification_report(y_test, y_pred, target_names=target_names)
print(class_report)

In [ ]:
y_pred_test = cv.predict(X_test['message'])
y_pred_train = cv.predict(X_train['message'])
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))
print('--------------------------------------------------')
print('\n',classification_report(y_train.values, y_pred_train, target_names=y.columns.values))

In [ ]:
for col in range(36):
    print(Y_test.columns[col])
    print(classification_report(y_test.iloc[:,col], y_pred[:,col]))
    print('-----------------------------------------------------')

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
cv.best_params_

### 9. Export your model as a pickle file

In [ ]:
with open('adaboost_cv.pkl', 'wb') as file:
    pickle.dump(cv_ada, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.